Перед запуском файл  "Log" нужно переименовать в файл "Log.csv"!

импортируем модули

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np

откроем логи

In [2]:
data = pd.read_csv('Log.csv', sep='\t', encoding='utf-8')

посмотрим на данные

In [3]:
data.head()

,timestamp,datetime,device,numdoc,region,request,urls
0,1520974800,2018-03-14 00:00,desktop,317731505,6,https://yandex.ru/search/?text=порно видео сек...,http://porno-sex-video.com/eroticheskie-filmy-...
1,1520974800,2018-03-14 00:00,desktop,277053268,191,https://yandex.ru/search/?text=sad story скача...,http://mp3party.net/music/8507899;http://muz-c...
2,1520974800,2018-03-14 00:00,touch,19987730,2,https://yandex.ru/search/?text=ольгино баня&fl...,http://www.bani.spb.ru/olgino.html;https://sau...
3,1520974800,2018-03-14 00:00,touch,196215754,26955,https://yandex.ru/search/?text=фирменный магаз...,http://www.marko.by/actions/;https://marko.vit...
4,1520974800,2018-03-14 00:00,desktop,3909882795,101704,https://yandex.ru/search/?text=порно&rl=313,http://porno365.xxx/;https://prostoporno.sex/c...


посмотрим количество запросов в файле

In [4]:
len(data)

1333503

выделим текстовую часть запросов

In [5]:
request = data['request']
requestList=[]
for i in range(len(request)):
    requestList.append(request[i].replace('https://yandex.ru/search/?text=', '').split('&')[0])

для удобства запишем в файл

In [6]:
f = open('1.txt', 'w')
for i in range(len(requestList)):    
    f.write(requestList[i].encode('utf8')+'\n')
f.close()

In [7]:
#подготовим данные к работе
dat = open('1.txt')
data = dat.readlines()
a, b= [], []
#создадим список запросов
for i in range(len(data)):
    a.append(data[i].strip().lower())
    
#создадим список списков запросов
import re
for j in range(len(a)):
    b.append(re.split(' ', a[j]))

Так как объем данных очень большой, не получится проанализировать все строки. Отфильтруем логи, в которых отсутсвуют ключевые слова, связанные с телевидением.  Поиск таких слов был осуществлен с помощью сервиса по подбору слов Яндекса. были введены запросы "телевидение" и "тв".

создание списка ключевых слов для фильтрации

In [8]:
filtrWords = '1канал антенна кабельное кабельный tv канал матч муз первый передач передача передачи подключить приставка программа прямая прямое прямом прямой рен ресивер россия1 ростелеком спутниковое спутниковый тв телевидение телевидения телевизионный телевизор телеканал триколор цифровое цифровой эфир эфирный нтв тв-3'
filtr = []
filtr.append(re.split(' ', filtrWords))

создадим множество из слов-фильтров

In [9]:
words = set()
for i in range(len(filtr)):
    for j in range(len(filtr[i])):
        words.add(filtr[i][j])

добавляем подходящие запросы в новый список

In [10]:
%%time 
newList=[]
for i in range(len(b)):
    for j in range(len(b[i])):
        if b[i][j] in words:
            newList.append(a[i])

Wall time: 1.51 s


оценим количество подходящих запросов

In [11]:
len(newList)

18842

оцением их долю в общем потоке

In [12]:
len(newList)/float(len(data))

0.014129701995421082

запишем подходящие запросы в файл

In [13]:
f = open('2.txt', 'w') 
for i in range(len(newList)):    
    f.write(newList[i]+'\n')
f.close()

Я бы разделил запросы на следующие группы:<p>
1) запрос на просмотр телевизора онлайн (аналог кнопки "ВКЛ" на пульте) - simpLookWords<p>
2) запрос на просмотр конкретного канала или программы телепередач(человек знает, что хочет смотреть) - lookChanWords<p>
3) запрос на тв-оборудование или провайдера (человек нуждается в услуге или товаре) - supportWords

In [14]:
simpLookWords = 'tv прямая прямое прямом прямой эфир'
lookChanWords = '1канал канал матч муз первый передач передача передачи рен россия1 телеканал триколор нтв тв-3'
supportWords = 'антенна кабельное кабельный подключить приставка программа ресивер ростелеком спутниковое спутниковый телевидение телевидения телевизионный телевизор цифровое цифровой эфирный'
#создадим списки
simpLook, lookChan, support = [], [], []
simpLook.append(re.split(' ', simpLookWords))
lookChan.append(re.split(' ', lookChanWords))
support.append(re.split(' ', supportWords))

In [15]:
#создадим множества
simpLookWords = set()
for i in range(len(simpLook)):
    for j in range(len(simpLook[i])):
        simpLookWords.add(simpLook[i][j])
lookChanWords = set()
for i in range(len(lookChan)):
    for j in range(len(lookChan[i])):
        lookChanWords.add(lookChan[i][j])
supportWords = set()
for i in range(len(support)):
    for j in range(len(support[i])):
        supportWords.add(support[i][j])

In [16]:
#готовим данные к работе
dat2 = open('2.txt')
data2 = dat2.readlines()
a2, b2= [], []
#создадим список запросов
for i in range(len(data2)):
    a2.append(data2[i].strip().lower())
    
#создадим список списков запросов
import re
for j in range(len(a2)):
    b2.append(re.split(' ', a2[j]))

In [17]:
#посчитаем вхождения
l_1 = 0
l_2 = 0
l_3 = 0
for i in range(len(b2)):
    for j in range(len(b2[i])):
        if b2[i][j] in simpLookWords:
            l_1 += 1
    for j in range(len(b2[i])):
        if b2[i][j] in lookChanWords:
            l_2 += 1
    for j in range(len(b2[i])):
        if b2[i][j] in supportWords:
            l_3 += 1

Получаем количество вхождений. Это не очень точный результат, так как в одном запросе могут быть ключевые слова из разных групп. Ключевые слова распределял по группам экспериментально.

In [18]:
print(l_1, l_2, l_3)

(11351, 10889, 6201)


Соотношение групп запросов: 1 : 0.959 : 0.546

Более точные результаты можно получить, анализируя косинусные расстояния между векторами вхождения слов в запрос. <p> Так как размер исходной матрицы очень большой, мощностей машины не хватает для их обработки (в один вектор обрабатывается больше 5 минут, с учетом их количества 1.3 млн дальнейший расчет нецелесообразен).

Вот как можно применить метод косинусных расстояний.

In [19]:
#Функция для создания словаря - мешка слов
def createBagWords(listAllwords):
    #создаем множество всех слов
    words = set()
    for i in range(len(listAllwords)):
        for j in range(len(listAllwords[i])):
            words.add(listAllwords[i][j])
    #создаем словарь, где ключ - порядковый номер, значение - слово из мешка
    keys = [i for i in range(len(words))]
    words = dict(zip(words, keys))
    return words

#функция на вход получает запрос, на выходе получаем вектор количества вхождений слов из мешка в наш запрос
def createVector(request):
    vector = np.zeros(len(words))
    for i in range(len(words)):
        if words.keys()[i] in request:
            vector[i]+=1
    return vector

Попробуем проанализировать таким методом уже отобранные логи (newList).

In [20]:
words = createBagWords(newList)

сфомируем матрицы из векторов вхождений

In [21]:
%%time
matrix = np.zeros((len(newList), len(words)))

for i in range(len(newList)):
    for j in range(len(words)):
        if words.keys()[j] in newList[i]:
            matrix[i][j] += 1

Wall time: 3.45 s


создадим вектор, с которым будем сравнивать вектора

In [22]:
normal = []
for i in range(len(filtrWords)):
    normal.append(filtrWords[i].strip().lower())
normal = createVector(normal)

In [23]:
from scipy.spatial.distance import cosine

Рассмотрим пары кос. расстояние - поисковый запрос.

In [24]:
for i in range(len(newList)):
    print cosine(matrix[i], normal), newList[i]

0.213281606163 смотреть онлайн матч футбол 1 прямая трансляция
0.213281606163 смотреть онлайн матч футбол 1 прямая трансляция
0.326366930291 рома шахтер прямая трансляция смотреть 13 марта
0.32056138714 пакет ночной триколор тв на 2018 год цена
0.32056138714 пакет ночной триколор тв на 2018 год цена
0.455418851363 программа передач
0.455418851363 программа передач
0.271383260595 второй канал онлайн прямой эфир россия 1
0.271383260595 второй канал онлайн прямой эфир россия 1
0.271383260595 второй канал онлайн прямой эфир россия 1
0.34807975948 матч футбол 1 iptv ссылка
0.404145976932 россия 1 прямой эфир
0.404145976932 россия 1 прямой эфир
0.429218207015 познавательное тв
0.911438511446 pro tv md gindeste liber
0.429218207015 первый канал
0.429218207015 первый канал
0.251518811435 не включается задняя передача октавия тур
0.534621079004 телеканал шалун
0.399754952001 режим спутника на смарт тв
0.511906469908 тв онлайн
0.451204527544 kaban tv online смотреть бесплатно
0.575562656186 матч

0.176806791316 акт приема передачи в пользование буровой установки для работы
0.575562656186 мачт тв
0.356880305716 телеканал россия 1 онлайн
0.194838951684 кто сейчас играет в футбол прямая трансляция
0.424350324399 матч футбол 2 онлайн
0.33114394594 первый поцелуй лестрейнджа
0.288942870863 секс со звёздами русского телевидения
0.616517505576 нтв
0.55443360566 шахтар рома 1 матч
0.207881965619 футбол 1 смотреть онлайн прямой эфир в хорошем качестве
0.207881965619 футбол 1 смотреть онлайн прямой эфир в хорошем качестве
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир
0.5553513541 tv1000 action программа
0.264852955885 можно ли подключить к музыкальному центру 4 колонки
0.392081241126 рома шахтер 13 марта 2018 прямая трансляция
0.264852955885 смотреть матч тв

0.271383260595 программа обучения востока сервиса
0.376082251894 смотреть тв канал хистори
0.376082251894 смотреть тв канал хистори
0.288942870863 про новости муз тв с гузель хасановой и мастанг
0.288942870863 про новости муз тв с гузель хасановой и мастанг
0.211106508344 передача бесхозяйных сетей ленинск- кузнецкий
0.380069580122 кабельное телевидение
0.380069580122 кабельное телевидение
0.578169256134 lg a290 передача mp3 по радио
0.312815729064 ростелеком личный кабинет
0.356880305716 дом кино программа на сегодня
0.511906469908 онлайн новий канал
0.312815729064 как сделать телевидение через интернет на смарт тв
0.312815729064 как сделать телевидение через интернет на смарт тв
0.268727575873 программа прошивки андроид через пк скачать на русском
0.424350324399 телевизор 4к что это
0.211106508344 нет на свете большего счастья чем услышать первый крик сына песня скачать бесплатно
0.511906469908 сиська тв
0.380069580122 передача про бегемотов
0.31938609027 запишите уравнение реакций п

0.251518811435 задняя передача на фольксваген пассат
0.288942870863 смотреть передачи дорожный патруль
0.511906469908 онлайн тв
0.921311052464 at the end of the day, all i want to do is go homie and water tv
0.455418851363 программа передач
0.455418851363 программа передач
0.232163954667 дом кино онлайн смотреть бесплатно прямой эфир
0.232163954667 дом кино онлайн смотреть бесплатно прямой эфир
0.334487735354 тв красная линия программа
0.334487735354 тв красная линия программа
0.288942870863 домашний смотреть онлайн прямой эфир
0.288942870863 домашний смотреть онлайн прямой эфир
0.251518811435 программа телепередач на сегодня российские каналы
0.326366930291 телевизионный кабель г.орша
0.380069580122 триколор приемники цены
0.539022777135 Компрессор цифровой 20л/мин 8А 12В tornado ac570 digital tornado bk54106
0.392081241126 программа передач на 5 канале за 4 марта 20128
0.392081241126 программа передач на 5 канале за 4 марта 20128
0.312815729064 программа телепередач на трк украина
0.

0.299377269172 из каких спиртов получен эфир c3h7-o-c5h11
0.356880305716 смотреть 1 канал онлайн
0.351796276448 наш спорт прямая трансляция биатлон ханты юниоры спринт мужчины 18 марта 2016
0.293693566304 программа утро россии на канале россия 1 сегодня смотреть онлайн от 14 марта 2018
0.334487735354 проверить скорость интернета ростелеком
0.240886883718 Сколько молодых 18летних в первый раз идут голосовать
0.310305333476 В основе разработки рабочей программы не лежит: ￼ Приказ Минобрнауки России № 277 от 25 марта 2015 года "О федеральном учебно-методическом объединении по общему образованию" ￼ Базисный учебный план ￼ Образовательная программа школы и учебный план ￼ Государственный образовательный стандарт (федеральный и региональный компонент)
0.312815729064 нтв плюс официальный сайт
0.36379098972 матч Севилья Манчестер Юнайтед
0.232163954667 ростелеком новосибирск официальный сайт личный кабинет
0.288942870863 приложение тинькофф для андроид как подключить карту
0.500865801515 2 кана

...

0.611077765869 ук тв
0.409832893477 программа школа 2100 отзывы
0.291799472172 программа для нарезки музыки
0.194838951684 как подключить видеокарту через райзер для майнинга
0.251518811435 первый губернатор белгородской губернии
0.35831105208 Не удается получить доступ к сайту Соединение сброшено. Попробуйте сделать следующее: Проверьте подключение к Интернету. Проверьте настройки прокси-сервера и брандмауэра. Выполните диагностику сети в windows Проверьте интернет-соединение. Проверьте подключение кабелей и перезапустите все маршрутизаторы и прочие используемые сетевые устройства. Разрешите opera доступ к сети с настройками брандмауэра или антивируса. Если программа входит в список тех, которым разрешен доступ к сети, удалите ее из списка и добавьте туда снова. Если вы используете прокси-сервер… Проверьте настройки соединения или свяжитесь с вашим сетевым администратором, чтобы убедиться, что прокси-сервер работает. Если вы уверены, что прокси-сервер вам не нужен: Перейдите в главное

0.356880305716 канал мир онлайн прямой эфир
0.192792647204 россия 1 2 часа смотреть онлайн прямой эфир екатеринбург
0.192792647204 россия 1 2 часа смотреть онлайн прямой эфир екатеринбург
0.488521841013 логотип канала dunya tv
0.380069580122 программа передач на сегодня
0.380069580122 программа передач на сегодня
0.380069580122 как найти канал в телеграмме
0.334487735354 как настроить телеканал сила тв на спутник
0.334487735354 как настроить телеканал сила тв на спутник
0.309731510037 нтв программа на сегодняшний день
0.309731510037 нтв программа на сегодняшний день
0.511906469908 тв онлайн
0.334487735354 пятница онлайн прямой эфир
0.334487735354 пятница онлайн прямой эфир
0.251518811435 горьковский телевизионный завод им ленина
0.232163954667 перспективы развития цифровой экономики в россии
0.260059926604 1+1 онлайн прямой эфир бесплатно в хорошем качестве
0.260059926604 1+1 онлайн прямой эфир бесплатно в хорошем качестве
0.245406225373 перечитай первый абзац текста какой была старушк

0.31938609027 программа телепередач российских каналов на 14.03.2018
0.34807975948 смотреть онлайн тв 1000 экшен
0.245406225373 нтв онлайн смотреть бесплатно прямой эфир в хорошем качестве
0.245406225373 нтв онлайн смотреть бесплатно прямой эфир в хорошем качестве
0.245406225373 нтв онлайн смотреть бесплатно прямой эфир в хорошем качестве
0.54267043962 торрент тв
0.386428008922 дом 2 за 11 марта 2018 ночной эфир
0.353238333236 матч инаш спортэа программа передач
0.353238333236 матч инаш спортэа программа передач
0.353238333236 матч инаш спортэа программа передач
0.380069580122 купить телевизор
0.251518811435 тв онлайн ртр канал прямой эфир сейчас
0.251518811435 тв онлайн ртр канал прямой эфир сейчас
0.251518811435 тв онлайн ртр канал прямой эфир сейчас
0.251518811435 тв онлайн ртр канал прямой эфир сейчас
0.232163954667 программа для редактирования музыки и втдео на телефон
0.326366930291 россия 24 смотреть онлайн прямой эфир
0.326366930291 россия 24 смотреть онлайн прямой эфир
0.87873

0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир
0.291799472172 челси барселона прямой эфир
0.291799472172 челси барселона прямой эфир
0.47678335643 канал футбол 2
0.575562656186 тв спас
0.279705942401 лч челси барселона 2018 смотреть онлайн прямой эфир бесплатно
0.279705942401 лч челси барселона 2018 смотреть онлайн прямой эфир бесплатно
0.54267043962 матч твЀ
0.32056138714 как подключить регистратор в cx-5
0.251518811435 мачт тв онлайн прямой эфир барса челси
0.251518811435 мачт тв онлайн прямой эфир барса челси
0.251518811435 мачт тв онлайн прямой эфир барса челси
0.380069580122 программа передач на сегодня
0.380069580122 программа передач на сегодня
0.211106508344 первый закон термодинамики необратимость процессов в природе 10 класс
0.891534771091 canli tv izle
0.334487735354 мтс прямой номер доставка
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв

0.232163954667 тв-3 смотреть онлайн прямой эфир
0.588335346289 МУЖСКОЕ КИНО программа передач
0.588335346289 МУЖСКОЕ КИНО программа передач
0.245406225373 матч арена смотреть онлайн прямой эфир бесплатно в хорошем качестве
0.245406225373 матч арена смотреть онлайн прямой эфир бесплатно в хорошем качестве
0.245406225373 матч арена смотреть онлайн прямой эфир бесплатно в хорошем качестве
0.36379098972 матч Севилья Манчестер Юнайтед
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир
0.511906469908 нтв онлайн
0.429218207015 главная дорога на нтв
0.455418851363 матч тв онлайн
0.455418851363 матч тв онлайн
0.309731510037 первый межпальцевой промежуток пульс
0.312815729064 твц онлайн прямой эфир
0.312815729064 твц онлайн прямой эфир
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир
0.356880305716 матч тв прямой эфир


Как видно, величина кос. расстояния уже не может дать нам никакой информации.<p> Это связано с тем, что мы отбирали эти запросы по фильтру, который использовали в нормали.<p>
Елси бы размер исходного файла был меньше, векторный анализ был бы успешен.